# TensorFlow Modelling
In this notebook, I did simple tensorflow model and I tried to embed but was not able to figure it out yet. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.sparse import SparseTensor
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint 
from sklearn.metrics import roc_auc_score, roc_curve 

Reading in the data.

In [3]:
X_train = pd.read_pickle('sparse_data.pkl')
y_train = pd.read_pickle('y_train.pkl')
X_test = pd.read_pickle('test_spasedata.pkl')
y_test = pd.read_pickle('y_test.pkl')

In [4]:
X_train.dtypes

vote_A     Sparse[int16, 0]
vote_N     Sparse[int16, 0]
vote_P     Sparse[int16, 0]
vote_Y     Sparse[int16, 0]
party_1    Sparse[int16, 0]
                 ...       
yukon      Sparse[int16, 0]
yy         Sparse[int16, 0]
zealand    Sparse[int16, 0]
érable     Sparse[int16, 0]
îles       Sparse[int16, 0]
Length: 5970, dtype: object

I want to get the data into a tensor but maintain it to be sparse.  I found a good piece of code to do so.

In [5]:
#Thanks stackoverflow : https://stackoverflow.com/questions/40896157/scipy-sparse-csr-matrix-to-tensorflow-sparsetensor-mini-batch-gradient-descent
def convert_sparse_matrix_to_sparse_tensor(coo):
    indices = np.mat([coo.row, coo.col]).transpose()
    return SparseTensor(indices, coo.data, coo.shape)

Converting it.

In [6]:
X_train_tensor = convert_sparse_matrix_to_sparse_tensor(X_train.sparse.to_coo())
X_test_tensor = convert_sparse_matrix_to_sparse_tensor(X_test.sparse.to_coo())

Running the model.

In [7]:
model = Sequential()
model.add(Dense(5970, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2500, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

I got an error from tensorflow saying I should do this, this sets up the data into canonical order which tensorflow needs for sparse data.

In [8]:
X_train_tensor = tf.sparse.reorder(X_train_tensor)
X_test_tensor = tf.sparse.reorder(X_test_tensor)

In [9]:
model.fit(X_train_tensor, np.array(y_train),
              batch_size = 128,
              epochs = 3,
              verbose = 1,
              validation_data = (X_test_tensor, np.array(y_test)))

Epoch 1/3
3838/3838 [==============================] - 1431s 373ms/step - loss: 0.5629 - accuracy: 0.6823 - val_loss: 0.5770 - val_accuracy: 0.7971
Epoch 2/3
3838/3838 [==============================] - 1431s 373ms/step - loss: 0.5362 - accuracy: 0.6977 - val_loss: 0.6952 - val_accuracy: 0.7914
Epoch 3/3
3838/3838 [==============================] - 1404s 366ms/step - loss: 0.5326 - accuracy: 0.6996 - val_loss: 0.7217 - val_accuracy: 0.7489


Saving the model.

In [14]:
model.save('saved_model/my_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [11]:
y_test.mean()

0.8334894150577242

### Embedding

I attempt to embed here but was not able to finish this, I may try again in the future.

In [32]:
df_train = pd.read_pickle('text_train.pkl')
df_test = pd.read_pickle('text_test.pkl')

In [33]:
df_train

,politician_id,votequestion_id,vote,dissent,vote_question_id,vote_date,description_en,id,name,party_id,riding_id,election_id,elected,election_date,byelection,votepercent
0,77,540,N,True,540,2008-05-26,that the bill be now read a second time and r...,77,John Cummins,1,59006,1,True,2008-10-14,False,55.75
1,77,534,N,False,534,2008-05-07,that the bill be now read a second time and r...,77,John Cummins,1,59006,1,True,2008-10-14,False,55.75
2,77,496,Y,False,496,2008-03-12,"that, in the opinion of the house, the governm...",77,John Cummins,1,59006,1,True,2008-10-14,False,55.75
3,77,490,Y,False,490,2008-03-05,that the bill be now read a second time and r...,77,John Cummins,1,59006,1,True,2008-10-14,False,55.75
4,77,461,Y,True,461,2007-12-12,that the bill be now read a second time and r...,77,John Cummins,1,59006,1,True,2008-10-14,False,55.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289087,9294,2366,N,False,2366,2015-06-17,"that, in the opinion of the house: ( a) the ...",9294,Jim Eglinski,1,48028,32,True,2015-10-19,False,72.30
289088,9294,2367,Y,False,2367,2015-06-17,that the 21st report of the standing committee...,9294,Jim Eglinski,1,48028,32,True,2015-10-19,False,72.30
289089,9294,2368,N,False,2368,2015-06-18,that the motion be amended by deleting all the...,9294,Jim Eglinski,1,48028,32,True,2015-10-19,False,72.30
289090,9294,2369,Y,False,2369,2015-06-18,that the bill be now read a third time and do ...,9294,Jim Eglinski,1,48028,32,True,2015-10-19,False,72.30


In [48]:
import nltk
from nltk import word_tokenize, sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
stpwrds = stopwords.words('english') 
punc = list(string.punctuation)

In [86]:
def cleaner(sentence):
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    return np.array(list([word for word in sentence.split() if word.lower() not in stpwrds]))

In [87]:
text_X1_train =  df_train['description_en']
text_X1_train = text_X1_train.apply(lambda x : cleaner(x))

In [75]:
text_X1_test = df_test['description_en']
text_X1_test = text_X1_test.apply(lambda x : cleaner(x))

In [92]:
max_length = text_X1_train.str.len().mean()
max_length = int(max_length*1.5)

In [88]:
text_X1_train

0         [bill, read, second, time, referred, standing,...
1         [bill, read, second, time, referred, standing,...
2         [opinion, house, government, stand, consistent...
3         [bill, read, second, time, referred, standing,...
4         [bill, read, second, time, referred, standing,...
                                ...                        
289087    [opinion, house, unemployment, rate, canada, r...
289088    [21st, report, standing, committee, procedure,...
289089    [motion, amended, deleting, words, word, “that...
289090                      [bill, read, third, time, pass]
289091                      [bill, read, third, time, pass]
Name: description_en, Length: 491172, dtype: object

In [95]:
all_words = []
for vote in text_X1_train:    
    for word in vote:
        if word not in all_words:
            all_words.append(word)

In [89]:
text_X1_train = pad_sequences(text_X1_train, maxlen = max_length,
                        padding = 'pre', truncating = 'pre', value = 0)

ValueError: invalid literal for int() with base 10: 'bill'

In [ ]:
x_valid = pad_sequences(x_valid, maxlen=max_review_length,
                        padding=pad_type, truncating=trunc_type, value=0)